In [2]:
import json 
import os
import logging 
import concurrent 
import asyncio 
import re 
import requests 

from pathlib import Path 
from itertools import chain 
from collections import namedtuple
from pprint import PrettyPrinter

from subgrounds import Subgrounds
from web3 import Web3
from concurrent.futures import ThreadPoolExecutor
from pycoingecko import CoinGeckoAPI

from prefect import flow, task
from prefect.tasks import task_input_hash
from prefect.filesystems import LocalFileSystem 
from prefect.orion.api.task_runs import read_task_run

import pandas as pd 
import numpy as np 
import altair as alt 
import missingno as miss

from flywheel_util.constants import (
    colors_24,
    colors_28, 
    addresses, 
    url_infura, 
    url_snapshot, 
    url_subgraphs, 
    snapshot_api_max_records_per_request, 
    snapshot_api_max_skip,
)
from flywheel_util.tasks import df_to_sql
from flywheel_util.utils import (
    ddf, 
    first_row, 
    camel_to_snake, 
    cg_get_market_history, 
    df_cols_camel_to_snake, 
    df_cols_change_prefix, 
    df_sort_cols, 
    remove_prefix, 
    graphql_execute, 
    remove_prefix, 
    remove_prefixes, 
    recursive_index_merge, 
    zip_dfs, 
    query_attrs, 
    compare_sets, 
    compare_cols, 
)
from flywheel_util.w3_utils import (
    get_verified_abi, 
    erc20_read_contract, 
    verified_contract
)
from sqlalchemy import create_engine
from sqlalchemy import text
engine = create_engine("sqlite+pysqlite:///votium_bribes.db", echo=True, future=True)

# logging.basicConfig(level=logging.INFO)

import logging
logging.basicConfig(level=logging.DEBUG)

# TODO: Figure out how this frax subgraph is useful
# https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxbp-subgraph/graphql

pp = PrettyPrinter().pprint

alt.data_transformers.disable_max_rows()

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/eth_utils/toolz.py:2: DeprecationWarning: The toolz.compatibility module is no longer needed in Python 3 and has been deprecated. Please import these utilities directly from the standard library. This module will be removed in a future release.
  from cytoolz import (
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/web3/_utils/normalizers.py:231: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(eth_abi.__version__) < LooseVersion("2"):


DataTransformerRegistry.enable('default')

In [3]:
w3 = Web3(Web3.HTTPProvider(url_infura))
cg = CoinGeckoAPI()

# Bribes Analysis 

We are interested in analyzing the impact of bribes performed by the frax protocol through votium. Specifically, we aim to measure their ability to incentivize liquidity. 

Votium is a marketplace bringing together individual holders of vlCVX and protocols who wish to aggregate vlCVX so that gauges of their preference can receive a larger weight and thus a larger share of crv emissions (as well as emissions from platforms like convex built on top of curve). 

This analysis is focused primarily on the bribing strategy as it relates to the various FraxBP metapools. 

<!-- The following metrics are of interest

- \\$ Bribes spent per pool (broken down by platform and incentive type)  
- \\$ Liquidity per pool (broken into Frax and non-Frax components) 

Once we have both the liquidity and bribe information, we can try to answer these questions
- Where are bribe dollars best spent? 
- How does the Frax protocol optimize its bribing strategy in order to maximize the amount of liquidity within it's pools?  -->

In [4]:
sg = Subgrounds()

sg_curve_pools = sg.load_subgraph(url_subgraphs.convex.curve_pools) 
sg_curve_vol = sg.load_subgraph(url_subgraphs.convex.curve_vol_mainnet)
sg_votium = sg.load_subgraph(url_subgraphs.votium.bribes) 

## Curve Liquidity for FraxBP + FraxBP Metapools

In [11]:
%load_ext autoreload
%autoreload 2

from flywheel_util.curve_liquidity_tasks import (
    query_curve_mpools_with_gauge, 
    query_curve_pool_snapshots,
    query_curve_pool_vol_snapshots,
    query_metapool_asset_ecosystem_volume,
    compute_pool_dfs,
    compute_curve_pool_reserves,
    join_curve_pool_vol,
    remove_inactive_pools,
    process_pool_snaps,
    process_metapool_snaps,
    remove_inactive_pools,
    process_pool_data,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'query_curve_mpools_with_gauge' and defined at '/Users/brycemorrow/Documents/code/frax-bribes/flywheel_util/src/flywheel_util/curve_liquidity_tasks.py:44' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'query_curve_pool_snapshots' and defined at '/Users/brycemorrow/Documents/code/frax-bribes/flywheel_util/src/flywheel_util/curve_liquidity_tasks.py:82' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'query_curve_pool_vol_snapshots' and defined at '/Users/brycemor

In [12]:
@flow(cache_result_in_memory=False)
def flow_fraxbp_metapool_data():
    # Retrieve data from api's / subgraphs 
    df_mpools_gauge = query_curve_mpools_with_gauge.submit()
    df_pool_snaps = query_curve_pool_snapshots.submit()
    df_pool_vol_snaps = query_curve_pool_vol_snapshots.submit()
        
    # df_pools table has one row per pool 
    # df_pool_coins table has one row per combination of pool and coin 
    df_pools, df_pool_coins = compute_pool_dfs(df_mpools_gauge, df_pool_snaps)

    # Remove inactive pools 
    df_pools, df_pool_coins, df_pool_snaps, df_pool_vol_snaps = remove_inactive_pools(
        df_pools, df_pool_coins, df_pool_snaps, df_pool_vol_snaps
    )
    
    # Dataframe containing only pool reserves. This separates pool level snapshot data 
    # from pool-coin level snapshot data. 
    df_reserves = compute_curve_pool_reserves(df_pool_snaps)
    
    # Compute pool snapshots. Contains metrics for all curve pools (not just metapools if we expand coverage) 
    df_pool_snaps = (
        df_pool_snaps
        .drop(columns=[
            'pool_coin_name', 'pool_name', 'pool_coin_address', 'pool_coin_decimals', 
            'reserves', 'reserves_usd', 'reserves_coin_price_usd'
        ])
        .drop_duplicates()
    )
    df_pool_snaps = join_curve_pool_vol(df_pool_snaps, df_pool_vol_snaps)
    df_pool_snaps = process_pool_snaps(df_pool_snaps)
    
    # We take our pool snapshots and augment them with extra info for our metapools 
    df_mpool_snaps = process_metapool_snaps(df_pools, df_pool_snaps, df_reserves)
    
    # finally, we add on ecosystem wide tvl to metapool snapshots 
    df_mpool_paired_tokens = df_pool_coins.loc[
        df_pool_coins.pool_coin_address != addresses.token.crvfrax
    ][['pool_coin_name', 'pool_coin_address']]
    token_addr_map = {name: addr for name, addr in df_mpool_paired_tokens.itertuples(index=False)}
    df_eco_vol_snaps = query_metapool_asset_ecosystem_volume(cg, token_addr_map)
    
    # Join the ecosystem wide tvl onto our metapool snaps 
    df_eco_vol_snaps = (
        # Adds pool address to the tvl for specific coins 
        df_eco_vol_snaps.merge(
            df_pool_coins[['pool_coin_name', 'pool_address']].drop_duplicates(), 
            how='left', 
            on='pool_coin_name'
        )
        [['date', 'pool_address', 'mpool_asset_eco_tvl_usd']]
    )
    assert not df_eco_vol_snaps.pool_address.isna().any()
    df_mpool_snaps = df_mpool_snaps.merge(df_eco_vol_snaps, how='left', on=['date', 'pool_address'])
    
    # Validation 
    num_metapools = len(df_mpool_snaps.pool_address.unique())
    num_metapools_gauge = len(df_pools.loc[~df_pools.pool_gauge.isna() & (df_pools.pool_fraxbp_metapool == True)])
    print(f"Discovered {num_metapools} metapools.")
    print(f"Number of pools with gauges: {num_metapools_gauge}")    
    
    for table_name, table_df in [
        ('pools', df_pools), 
        ('pool_coins', df_pool_coins), 
        ('reserves', df_reserves), 
        ('pool_snaps', df_pool_snaps),  
        ('mpool_snaps', df_mpool_snaps), 
    ]:
        df_to_sql.submit(engine, table_df, table_name)
    
    return df_pools, df_pool_coins, df_reserves, df_pool_snaps, df_mpool_snaps 


/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/flows.py:205: UserWarning: A flow named 'flow-fraxbp-metapool-data' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_23829/1071678596.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [13]:
df_pools, df_pool_coins, df_reserves, df_pool_snaps, df_mpool_snaps = flow_fraxbp_metapool_data()

09:01:17.063 | INFO    | prefect.engine - Created flow run 'accomplished-jerboa' for flow 'flow-fraxbp-metapool-data'

09:01:17.122 | INFO    | Flow run 'accomplished-jerboa' - Created task run 'query_curve_pool_snapshots-2122ee9e-0' for task 'query_curve_pool_snapshots'

09:01:17.123 | INFO    | Flow run 'accomplished-jerboa' - Submitted task run 'query_curve_pool_snapshots-2122ee9e-0' for execution.

09:01:17.132 | INFO    | Flow run 'accomplished-jerboa' - Created task run 'query_curve_pool_vol_snapshots-76169dd1-0' for task 'query_curve_pool_vol_snapshots'

09:01:17.133 | INFO    | Flow run 'accomplished-jerboa' - Submitted task run 'query_curve_pool_vol_snapshots-76169dd1-0' for execution.

...

2022-12-15 09:01:17,937 INFO sqlalchemy.engine.Engine COMMIT


09:01:17.937 | INFO    | sqlalchemy.engine.Engine - COMMIT

09:01:17.946 | INFO    | Task run 'df_to_sql-2e8b9530-1' - Finished in state Completed()

09:01:17.968 | INFO    | Flow run 'accomplished-jerboa' - Finished in state Completed()

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2a1304700>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2a09050c0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2a13051e0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2a0907dc0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x29d4aae00>


In [71]:
# TODO: Show top 19 and aggregate others into single "Other" category.
d_mpool_share = (
    df_mpool_snaps[[
        'date', 'pool_address', 'crvfrax_in_mpool', 'crvfrax_in_all_mpools', 'crvfrax_share_mpools', 'crvfrax_share_fraxbp'
    ]]
    .merge(df_pools[['pool_address', 'pool_name']], how='left', on='pool_address')
)
d_mpool_share_last = d_mpool_share.loc[d_mpool_share.date == d_mpool_share.date.max()]

### (Chart) Metapool TVL Share 

- Segmented by pool name 

In [72]:
# Table of most recent values 
# d = df_bp_share_last.copy()
# d.snapshot_bp_lp_metapool_share = d.snapshot_bp_lp_metapool_share * 100
# ddf(d.sort_values('snapshot_bp_lp_metapool_share', ascending=False).reset_index(drop=True))

x = alt.X('date:T', title="date")
color = alt.Color("pool_name:N", scale=alt.Scale(range=colors_24))

chart_share_of_fraxbp = (
    alt.Chart(d_mpool_share)
    .mark_area()
    .encode(
        x=x, 
        y=alt.Y('crvfrax_share_fraxbp:Q', axis=alt.Axis(format=",%", title="% FraxBP")), 
        color=color, 
        tooltip=["pool_name:N", alt.Tooltip('crvfrax_share_fraxbp:Q', format=".1%", title='% FraxBP')]
    )
    .properties(title="Historical Metapool % FraxBP")
) 
chart_share_of_fraxbp_current = (
    alt.Chart(d_mpool_share_last)
    .mark_arc()
    .encode(
        theta='crvfrax_share_fraxbp:Q', 
        color=color, 
        tooltip=["pool_name:N", alt.Tooltip('crvfrax_share_fraxbp:Q', format=".1%", title='% FraxBP')]
    )
    .properties(title="Current Metapool % FraxBP")
)

chart_metapool_share = (
    alt.Chart(d_mpool_share)
    .mark_area()
    .encode(
        x=x, 
        y=alt.Y('crvfrax_share_mpools:Q', axis=alt.Axis(format=",%", title="% Across Metapools"), scale=alt.Scale(domain=[0,1])), 
        color=color, 
        tooltip=["pool_name:N", alt.Tooltip('crvfrax_share_mpools:Q', format=".1%", title='% Across Metapools')]
    )
    .properties(title="Historial Metapool Share Across All Metapools")
)

(chart_share_of_fraxbp | chart_share_of_fraxbp_current | chart_metapool_share)

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [73]:
mpool_addrs = df_pools.loc[df_pools.pool_fraxbp_metapool == True].pool_address.unique()
df_tvl = (
    # Get share of non crvFRAX in each of the metapools 
    df_reserves.loc[
        df_reserves.pool_address.isin(mpool_addrs) & (df_reserves.pool_coin_address != ADDRESS_CRVFRAX)
    ]
    .merge(df_pools[['pool_address', 'pool_type', 'pool_name']], how='left', on='pool_address')
)

x = alt.X("date:T", axis=alt.Axis(title="Timestamp"))
color = alt.Color("pool_name:N", scale=alt.Scale(range=colors_28))
facet = alt.Facet('pool_type:N', columns=1, header=alt.Header(title=None, labels=False))

# Charts 
chart_tvl_type_breakdown = (
    alt.Chart(df_tvl)
    .transform_aggregate(groupby=['date', 'pool_type'], tvl_pool_type="sum(reserves_usd)")
    .transform_joinaggregate(groupby=['date'], tvl_total="sum(tvl_pool_type)")
    .encode(
        x=x, 
        tooltip=[
            "date:T", 
            "pool_type:N",
            alt.Tooltip("tvl_pool_type:Q", format="$,d"), 
            alt.Tooltip("tvl_total:Q", format="$,d")
        ]
    )
)
chart_tvl_type_breakdown_area = (
    chart_tvl_type_breakdown
    .mark_area()
    .encode(
        y=alt.Y("tvl_pool_type:Q", axis=alt.Axis(title="TVL ($)")), 
        color="pool_type:N",
    )
)
chart_tvl_type_breakdown_line = (
    chart_tvl_type_breakdown
    .mark_line()
    .encode(y="tvl_total:Q")
)

alt.vconcat(
    alt.hconcat(
        (
            alt.Chart(df_tvl)
            .mark_area()
            .encode(
                x=x,
                y=alt.Y("reserves_usd:Q", axis=alt.Axis(title="TVL ($)")), 
                facet=facet, 
                color=color, 
                tooltip=[
                    "date:T", 
                    "pool_name:N", 
                    alt.Tooltip("reserves_usd:Q", format="$,d")
                ]
            )
            .resolve_scale(y="independent").resolve_axis("independent")
            .properties(title="Historical Metapool TVL Breakdown")
        ), 
        (
            alt.Chart(
                df_tvl.loc[df_tvl.date == df_tvl.date.max()]
            )
            .mark_arc()
            .encode(
                theta="reserves_usd:Q", 
                color=color, 
                facet=facet, 
                tooltip=[
                    "pool_name:N", 
                    alt.Tooltip("reserves_usd:Q", format="$,d", title="TVL"),
                ] 
            )
            .resolve_scale(theta="independent")
            .properties(title="Current Metapool TVL Breakdown")
        )
    ),
    alt.layer(chart_tvl_type_breakdown_area, chart_tvl_type_breakdown_line), 
    center=True
).resolve_legend(color="independent").resolve_scale(color="independent")

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

### Snapshot Proposals 

We retrieve all snapshot proposals for convex gauge weight snapshots. 

Since votium bribes are intended to get vlCVX holders to vote for particular choices in this snapshot, this data is necessary. 

In [74]:
cmd = '''line {0}
line {1}
line {2}'''.format(1,2,3)

print(cmd)

line 1
line 2
line 3


In [107]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport


async def graphql_execute(
    url, 
    query, 
    variable_values=None, 
    paginate=False, 
    page_size=1000, 
    page_size_variable='page_size', 
    page_offset_variable='page_offset',
    verbose=False, 
    batch_size=10000, 
    poll_secs=2, 
):
    # Batch size must be a multiple of page size 
    assert batch_size % page_size == 0
    variable_values = variable_values or {}
    transport = AIOHTTPTransport(url=url)
    gquery = gql(query)
    query_name = gquery.to_dict()['definitions'][0]['name']['value'].lower()
    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(transport=transport, fetch_schema_from_transport=True) as session:
        if not paginate:
            res = await session.execute(gquery, variable_values=variable_values)
            records = res[query_name]
            if verbose: 
                print(f"Query returned {len(records)} records with page size {page_size}.")
            return records 
        else: 
            results = []
            finished = False 
            i = 0
            n_pages = int(batch_size / page_size)
            
            async def _request(page_num):
                # Requests the page_num-th page of data 
                res = await session.execute(
                    gquery, variable_values={
                        **variable_values, 
                        page_offset_variable: page_size * page_num, 
                        page_size_variable: page_size
                    }
                )
                records = res[query_name]
                if verbose: 
                    print(f"-- Page {page_num} returned {len(records)} records with page size {page_size}.")
                return records
            
            while not finished: 
                if i != 0: 
                    await asyncio.sleep(poll_secs)
                # Queue a sequence of batch_size / page_size requests, each requesting page_size records 
                """
                Paging example: 
                ---------------
                page_size = 150
                page_start = 1
                page_end = 3 
                0: 0-149
                1: 150-299
                2: 300-449
                3: 450-599
                """
                page_start = i 
                page_end = i + n_pages - 1 
                record_start = page_start * page_size
                record_end = (page_end + 1) * page_size - 1
                if verbose: 
                    print(f"Requesting page range {page_start} - {page_end} / Record Range [{record_start}, {record_end}]")
                futures = [_request(j) for j in range(page_start, page_end + 1)]
                list_records = await asyncio.gather(*futures)
                finished = any(len(records) == 0 for records in list_records)
                results.extend(list_records) 
                i = page_end + 1 
            return list(chain(*results))


@task(persist_result=True, cache_key_fn=lambda *args: "snapshot_proposals4")
async def query_snapshot_proposals(): 
    """Get all snapshot proposals corresponding to convex gauge weight votes. 
    
    Within each gauge weight vote, holders of vlCVX collectively determine how convex should distribute it's 
    veCRV in subsequent curve gauge votes. 
    
    - Votes are pulled from the snapshot graphql endpoint. 
    """
    proposal_attrs = ['id', 'title', 'choices', 'start', 'end', 'state', 'votes', 'state']
    proposals = await graphql_execute(
        URL_SNAPSHOT, 
        '''
        query Proposals {
          proposals(
            first: <first>,
            where: {space: "cvx.eth"},
            orderBy: "created",
            orderDirection: asc
          ) {
            <proposal_attrs>
          }
        }
        '''
        .replace('<proposal_attrs>', '\n'.join(proposal_attrs))
        .replace('<first>', str(snapshot_api_max_records_per_request))
    )
    # Number of rounds here should match number of rounds on llama airforce 
    # https://llama.airforce/#/bribes/rounds/votium/cvx-crv/
    df_proposals = pd.DataFrame(proposals)
    df_proposals.title = df_proposals.title.str.lower()
    df_proposals = df_proposals.loc[~df_proposals.title.str.startswith("(test)") & df_proposals.title.str.contains("gauge weight")]
    df_proposals.start = pd.to_datetime(df_proposals.start, unit='s').dt.date
    df_proposals.end = pd.to_datetime(df_proposals.end, unit='s').dt.date 
    df_proposals = df_proposals.sort_values('start').reset_index(drop=True)
    # On votium, the proposal id is keccak256 hashed. So we need to perform this operation on the data pulled from snapshot 
    df_proposals['id_keccak256'] = df_proposals.id.apply(
        lambda _id: Web3.keccak(text=_id).hex() if not _id.startswith('0x') else Web3.keccak(hexstr=_id).hex()
    )
    # Validate that this query pulls in all data by ensuring that the number of votium voting rounds we get from this query 
    # matches the expected number of voting rounds, which we can compute using simple timedelta logic, the date of the 
    # first vote, and the current date. 
    dmin = pd.Timestamp('2021-09-16') # day of the first votium snapshot 
    assert pd.Timestamp(df_proposals.start.min()) == dmin
    actual_dates = df_proposals.sort_values('start')['start']
    expected_dates = pd.Series([d.date() for d in pd.date_range(pd.Timestamp('2021-09-16'), pd.Timestamp.now(), freq='14D')])
    pd.testing.assert_series_equal(actual_dates.iloc[:len(expected_dates)], expected_dates, check_names=False)
    # Sometimes, snapshot proposals for a week are posted the night before the week begins. This allows some lee-way 
    assert abs(len(actual_dates) - len(expected_dates)) <= 1
    df_proposals = df_proposals.rename(columns={
        'id': 'proposal_id', 
        'title': 'proposal_title', 
        'choices': 'proposal_choices', 
        'start': 'proposal_start', 
        'end': 'proposal_end', 
        'id_keccak256': 'proposal_id_keccak256', 
        'votes': 'vote_count', 
        'state': 'proposal_state', 
    })
    df_proposals = df_proposals.sort_values('proposal_start').reset_index(drop=True).reset_index()
    df_proposals['proposal_round'] = df_proposals['index'] + 1 
    df_proposals = df_proposals.drop(columns=['index', 'state']) 
    return df_proposals 
    
@task(persist_result=True, cache_key_fn=lambda context, pid: f"snapshot_proposal_votes1_{pid}", tags=['network_request'])
async def blocking_query_snapshot_votes_for_proposal(pid: str): 
    return await graphql_execute(
        url_snapshot, 
        '''
            query Votes($page_size: Int!, $page_offset: Int!, $pid: String!) {
              votes (
                first: $page_size
                skip: $page_offset
                where: { proposal: $pid }
              ) {
                proposal { id }
                id
                voter
                created
                choice
                vp
                vp_state
              }
            }
        ''', 
        paginate=True, 
        page_size=snapshot_api_max_records_per_request,
        batch_size=snapshot_api_max_skip + snapshot_api_max_records_per_request, 
        variable_values={'pid': pid}, 
        verbose=True, 
    )
    
@task
async def process_snapshot_votes(votes, verbose=True): 
    """Get all votes for all convex gauge weight snapshot proposals. 
        
    - Votes are pulled from the snapshot graphql endpoint. 
    """
    vote_records = []
    for v in votes: 
        for index, amount in v['choice'].items(): 
            r = {
                **v, 
                'choice_index': int(index) - 1, # convert 1 to 0 based indexing for join with other data later 
                'choice_weight': amount, 
                'is_votium': v['voter'].lower() == VOTIUM_VOTER
            }
            del r['choice'] 
            vote_records.append(r)
    df_votes = pd.DataFrame(vote_records)
    df_votes['proposal_id'] = df_votes.proposal.apply(lambda v: v['id']) 
    # So choice weights are really weird on snapshot. The best way to normalize is to find all votes 
    # by a voter in a proposal, sum the weights to get a total, then use this to find the fraction of 
    # the users total voting power (vp) allocated to the choices they voted for. 
    df_votes['choice_weight_total'] = df_votes.groupby(['proposal_id', 'voter'])['choice_weight'].transform('sum')
    df_votes['choice_percent'] = df_votes.choice_weight / df_votes.choice_weight_total * 100
    df_votes['choice_vp'] = (df_votes.choice_percent / 100) * df_votes.vp 
    df_votes = df_votes.drop(columns=['proposal', 'choice_weight_total', 'choice_weight']) 
    df_votes = df_votes.rename(columns={'created': 'vote_created', 'vp': 'total_vp', 'id': 'vote_id'})
    return df_votes

@task(persist_result=True, cache_key_fn=lambda *args: "gauge_results6")
def query_gauge_info(): 
    # Most recent gauge information. We will map the name of all the current gauges to the names 
    # within the snapshot proposal, so we can determine which choices in the proposals correspond 
    # to which gauges. 
    gauge_data = requests.get('https://api.curve.fi/api/getAllGauges').json()
    df_gauges = pd.DataFrame([
        {
          'gauge_name': d['name'].lower(), 
          'gauge_short_name': d['shortName'].lower(), 
          'gauge_address': d['gauge']
        } 
        for _, d in gauge_data['data'].items() 
    ])
    return df_gauges 

@task(persist_result=True, cache_key_fn=lambda *args: 'votium_epoches2')
def query_votium_epoches(): 
    # Get all votium voting epochs. Once we have validated that this set of epochs matches our set of snapshot proposals, 
    # we need to merge this data with our snapshot proposal data. 
    query_epoches = sg_votium.Query.epoches(first=1000, orderBy="initiatedAt", where={"bribeCount_gt": 0})
    df_epoches = query_attrs(sg, query_epoches, ['id', 'initiatedAt', 'deadline'])
    df_epoches = df_cols_change_prefix(df_epoches, 'epoches', 'epoch')
    df_epoches = df_cols_camel_to_snake(df_epoches) 
    df_epoches['epoch_start_date'] = pd.to_datetime(df_epoches.epoch_initiated_at, unit="s").dt.date
    df_epoches['epoch_end_date'] = pd.to_datetime(df_epoches.epoch_deadline, unit="s").dt.date 
    return df_epoches 

@task(persist_result=True, cache_key_fn=lambda *args: 'votium_bribes2')
def query_votium_bribes(epoch_ids): 
    # https://github.com/convex-community/convex-subgraph/blob/main/subgraphs/votium/src/mapping.ts
    # Addresses associated with the frax protocol used for votium bribes 
    # TODO: Frax controls some subset of the TVL in it's liquidity pools. Need to be cognizant of this because it leads 
    #       to a rebate the lowers the cost of bribing. 
    # TODO: Frax's vlCVX is not custodied in the investor custodian wallet. 
    token_map = {ADDRESS_FXS: 'FXS', ADDRESS_FRAX: 'FRAX'}
    query_bribes = sg_votium.Query.bribes(first=100000, where={
        "epoch_in": epoch_ids, "token_in": [ADDRESS_FXS, ADDRESS_FRAX]
    })
    df_bribes = query_attrs(sg, query_bribes, ['id', 'amount', 'token', 'choiceIndex', 'epoch.id'])
    df_bribes = df_cols_change_prefix(df_bribes, "bribes", "bribe")
    df_bribes = df_cols_camel_to_snake(df_bribes)
    df_bribes['bribe_token_name'] = df_bribes.bribe_token.apply(lambda addr: token_map[addr])
    df_bribes['bribe_tx_hash'] = df_bribes.bribe_id.apply(lambda _id: _id.split('-')[0])
    df_bribes.bribe_amount /= 1e18 # both frax and fxs have 18 decimals
    df_bribes = df_bribes.drop(columns=['bribe_id'])
    return df_bribes 

@task(persist_result=True, cache_key_fn=lambda *args: 'bribe_asset_prices2')
def query_bribe_asset_prices():
    token_addr_map = {'frax': ADDRESS_FRAX, 'fxs': ADDRESS_FXS}
    df_prices = cg_get_market_history(cg, token_addr_map, include_volume=False, include_market_cap=False)
    df_prices = (
        df_prices
        .drop(columns='date')
        .pivot(index="timestamp", columns="token_name", values="prices")
        .reset_index().rename(columns={'frax': 'price_frax', 'fxs': 'price_fxs'})
    )
    return df_prices 

@task(persist_result=True, cache_key_fn=lambda *args: "votium_reclaims2") 
def query_votium_claims():
    # Query claim events from the votium multi-merkle stash 
    address_votium_multi_merkle_stash = '0x378ba9b73309be80bf4c2c027aad799766a7ed5a'
    contract = verified_contract(address_votium_multi_merkle_stash)
    addrs = {
        '0xb1748c79709f4ba2dd82834b8c82d4a505003f27': "investor custodian", 
        '0x7038c406e7e2c9f81571557190d26704bb39b8f3': "utility contract maker", 
    }
    events_claimed = []
    for account in addrs.keys(): 
        filter_claims = contract.events.Claimed.createFilter(
            fromBlock=13320169, argument_filters={"account": account}
        )
        _events_claimed = filter_claims.get_all_entries()
        print(f"Found {len(_events_claimed)} `Claimed` events on votium multi merkle stash for account {account}") 
        events_claimed.extend(_events_claimed)
    return events_claimed 

@task 
def process_events_claimed(events_claimed):
    df_claims = pd.DataFrame([
        {'token': e.args.token, 'account': e.args.account, 'amount': e.args.amount, "block_number": e.blockNumber} 
        for e in events_claimed
    ])
    df_claims = df_claims.loc[df_claims.token != '0xa693B19d2931d498c5B318dF961919BB4aee87a5'] # UST lol 
    # abi for DAI but works for all these erc20's since we're just calling symbol on the contract. TUSD is a proxy for some reason 
    # so if we try to auto-pull the abi using etherscan we get the wrong one.
    abi = '[{"inputs":[{"internalType":"uint256","name":"chainId_","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"src","type":"address"},{"indexed":true,"internalType":"address","name":"guy","type":"address"},{"indexed":false,"internalType":"uint256","name":"wad","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":true,"inputs":[{"indexed":true,"internalType":"bytes4","name":"sig","type":"bytes4"},{"indexed":true,"internalType":"address","name":"usr","type":"address"},{"indexed":true,"internalType":"bytes32","name":"arg1","type":"bytes32"},{"indexed":true,"internalType":"bytes32","name":"arg2","type":"bytes32"},{"indexed":false,"internalType":"bytes","name":"data","type":"bytes"}],"name":"LogNote","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"src","type":"address"},{"indexed":true,"internalType":"address","name":"dst","type":"address"},{"indexed":false,"internalType":"uint256","name":"wad","type":"uint256"}],"name":"Transfer","type":"event"},{"constant":true,"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"PERMIT_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"usr","type":"address"},{"internalType":"uint256","name":"wad","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"usr","type":"address"},{"internalType":"uint256","name":"wad","type":"uint256"}],"name":"burn","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"guy","type":"address"}],"name":"deny","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"usr","type":"address"},{"internalType":"uint256","name":"wad","type":"uint256"}],"name":"mint","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"src","type":"address"},{"internalType":"address","name":"dst","type":"address"},{"internalType":"uint256","name":"wad","type":"uint256"}],"name":"move","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"holder","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"nonce","type":"uint256"},{"internalType":"uint256","name":"expiry","type":"uint256"},{"internalType":"bool","name":"allowed","type":"bool"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"usr","type":"address"},{"internalType":"uint256","name":"wad","type":"uint256"}],"name":"pull","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"usr","type":"address"},{"internalType":"uint256","name":"wad","type":"uint256"}],"name":"push","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"guy","type":"address"}],"name":"rely","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"dst","type":"address"},{"internalType":"uint256","name":"wad","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"src","type":"address"},{"internalType":"address","name":"dst","type":"address"},{"internalType":"uint256","name":"wad","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"wards","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}]'
    token_symbol_map = {}
    token_decimal_map = {}
    for a in df_claims.token.unique(): 
        contract = w3.eth.contract(a, abi=abi)
        token_symbol_map[a] = contract.functions.symbol().call()
        token_decimal_map[a] = contract.functions.decimals().call()
    df_claims['token_name'] = df_claims.token.apply(lambda a: token_symbol_map[a])
    df_claims['token_decimals'] = df_claims.token.apply(lambda a: token_decimal_map[a])
    df_claims['amount'] = df_claims.apply(lambda row: row['amount'] / 10** row['token_decimals'], axis=1) 
    # Get the timestamp of a block 
    block_timestamp_map = {bnum: w3.eth.get_block(int(bnum))['timestamp'] for bnum in df_claims.block_number.unique()}    
    df_claims['timestamp'] = df_claims.block_number.apply(lambda block_num: block_timestamp_map[block_num])
    df_claims.timestamp = pd.to_datetime(df_claims.timestamp, unit='s')  
    return df_claims 

@task 
def validate_proposals_vs_epoches(df_proposals, df_epoches): 
    """We pull epoches from the votium subgraph, and snapshot proposals from the snapshot api. 
    
    We need to ensure that we have a one to one mapping between these two entites. 
    Additionally, we remove one erroneous proposal from our set of epoches 
    """
    # Validate that the epoch dates for votium bribes match the proposal data we pulled from snapshot. `
    epoch_dates = df_epoches.epoch_start_date.unique().tolist()
    proposal_dates = df_proposals.proposal_start.unique().tolist()
    d_exclude = pd.Timestamp('2021-11-08').date()
    assert d_exclude in epoch_dates and not d_exclude in proposal_dates
    epoch_dates.remove(d_exclude)
    assert set(epoch_dates) == set(proposal_dates)
    df_epoches = df_epoches.loc[df_epoches.epoch_start_date != d_exclude].reset_index(drop=True)
    return df_epoches

@task 
def process_df_epoches(df): 
    df = (
        df.sort_values('epoch_start_date').reset_index(drop=True).reset_index().rename(columns={'index': 'proposal_round'})
    )
    df.proposal_round += 1
    df = df.drop(columns=['epoch_initiated_at', 'epoch_deadline'])
    df.epoch_start_date = pd.to_datetime(df.epoch_start_date)
    df.epoch_end_date = pd.to_datetime(df.epoch_end_date)
    return df 

@task(cache_key_fn=task_input_hash)
def label_bribes(df_bribes): 
    # Label bribes from known frax associated addresses 
    def get_tx_from(tx_hash): 
        # Determine the "from" address for the transaction that contained the bribe 
        return w3.eth.get_transaction(tx_hash)['from'] 

    # compute mapping of tx_hash to the from address of the transaction 
    tx_hashes = df_bribes.bribe_tx_hash.unique()
    tx_from_map = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {tx_hash: executor.submit(get_tx_from, tx_hash) for tx_hash in tx_hashes}
        for tx_hash, future in futures.items(): 
            tx_from_map[tx_hash] = future.result()

    frax_bribe_addresses = [
        # ('comptroller', '0xb1748c79709f4ba2dd82834b8c82d4a505003f27'),
        # ('cvx locker amo', '0x7038c406e7e2c9f81571557190d26704bb39b8f3'),
        ('investor custodian', '0x5180db0237291A6449DdA9ed33aD90a38787621c'),
        ('frax1.eth', '0x234D953a9404Bf9DbC3b526271d440cD2870bCd2'),
    ]
    address_labels = {addr.lower(): name for name, addr in frax_bribe_addresses}

    df_bribes['bribe_from'] = df_bribes['bribe_tx_hash'].apply(lambda tx_hash: tx_from_map[tx_hash])
    df_bribes['briber_label'] = df_bribes['bribe_from'].apply(lambda a: address_labels.get(a.lower(), 'unknown'))

    label_counts = df_bribes.briber_label.value_counts(dropna=False)
    print("Label count for addresses submitting bribes") 
    print(label_counts)

    return df_bribes 
        
@task 
def join_bribes_choices_epoches_prices(df_bribes, df_choices, df_epoches, df_prices): 
    df = (
        # Aggregate bribes by unique combination of proposal, briber, bribe token, bribe choice index 
        df_bribes.groupby([
            'bribe_epoch_id', 'bribe_choice_index', 'bribe_token', 'bribe_token_name', 'bribe_from', 'briber_label'
        ])
        ['bribe_amount'].sum().reset_index()
    )
    plen = len(df)
    df = (
        df
        # Join in epoch, so we know the time period of voting (for visualizing a time axis) 
        .merge(
            df_epoches[['proposal_round', 'epoch_id', 'epoch_start_date', 'epoch_end_date']].rename(columns={'epoch_id': 'bribe_epoch_id'}), 
            how='left', 
            on='bribe_epoch_id', 
            validate='m:1'
        )
        # Join in choice information, so we know what was bribed for 
        .merge(
            df_choices[['proposal_round', 'choice', 'choice_index']].rename(columns={'choice_index': 'bribe_choice_index', 'choice': 'bribe_choice'}), 
            how='left',
            on=['proposal_round', 'bribe_choice_index'],
            validate='m:1',
        )
    )
    assert plen == len(df)

    # Append price data for FRAX and FXS so we can denominate bribes in $
    # For pricing, we use the price of the assets on the day the bribing period ends. 
    # If the bribing period is still active, then we use the most recent price. 
    df['price_timestamp'] = df.epoch_end_date
    df.loc[df.price_timestamp >= df_prices.timestamp.max(), 'price_timestamp'] = df_prices.timestamp.max()
    df = df.sort_values('price_timestamp').reset_index(drop=True)
    df = pd.merge_asof(df, df_prices, left_on="price_timestamp", right_on="timestamp")
    df['bribe_amount_usd'] = None 
    mask_fxs = df.bribe_token_name == 'FXS'
    mask_frax = df.bribe_token_name == 'FRAX'
    df.loc[mask_fxs, 'bribe_amount_usd'] = df.loc[mask_fxs].bribe_amount * df.loc[mask_fxs].price_fxs
    df.loc[mask_frax, 'bribe_amount_usd'] = df.loc[mask_frax].bribe_amount * df.loc[mask_frax].price_frax
    df = df.drop(columns=['price_timestamp', 'price_frax', 'price_fxs', 'timestamp'])

    df = df_sort_cols(df, ['proposal', 'bribe', 'choice', 'epoch']) 
    
    return df 
    
@task 
def standardize_choices(df_votium_frax, df_gauges): 
    # Goal: For each choice in the snapshot proposal (i.e. a curve pool gauge), we want to pair the 
    #       choice name taken from the snapshot API with the address of the curve pool that choice 
    #       corresponds to. 
            
    # Remove irrelevant gauges  
    remove_choices = [
        "arbitrum-f-4pool", # 4pool never launched 
        "tusd", # Only bribed for in 1 round. Not really sure what to do about this one 
        "ypool", # Single tiny bribe in round 13, not sure what this is.  
    ]
    df_votium_frax = df_votium_frax.loc[~df_votium_frax.bribe_choice.isin(remove_choices)]
    
    # We get the current set of gauges from the curve API. These names are what appear in the snapshot proposal 
    # This will get the most recent set of names but the structure of the names has changed over time. More work 
    # needs to be done to account for historical differences in naming pools 
    canonical_choices = {
        l[0]: l[1] for l in [
            ['frax', 'frax+3crv (0xd632…)'], 
            ['d3pool', 'frax+fei+alusd (0xbaaa…)'], 
            ['fpifrax', 'frax+fpi (0xf861…)'], 
            ['2pool-frax', 'frax+usdc (0xdcef…)'], 
            ['fraxbpsusd', 'susd+fraxbp (0xe3c1…)'], 
            ['fraxbplusd', 'lusd+fraxbp (0x497c…)'], 
            ['fraxbpbusd', 'busd+fraxbp (0x8fdb…)'], 
            ['fraxbpape', 'apeusd+fraxbp (0x04b7…)'], 
            ['fraxbpalusd', 'alusd+fraxbp (0xb30d…)'], 
            ['fraxbpusdd', 'usdd+fraxbp (0x4606…)'], 
            ['fraxbptusd', 'tusd+fraxbp (0x33ba…)'], 
            ['fraxbpgusd', 'gusd+fraxbp (0x4e43…)'], 
        ]
    }
    
    def preprocess_choice(choice):
        choice = choice.lower()
        # Some older voting rounds used crvfrax while newer ones use fraxbp 
        choice = choice.replace('crvfrax', 'fraxbp')
        # Some older voting rounds prefixed factory pools with f- while newer rounds do not
        if choice.startswith("f-"): 
            choice = choice[2:]
        # Some older voting rounds showed addresses in form (0x...ab123)
        # whereas newer rounds use the form (0x...) without trailing values. 
        # Here, we remove trailing values if they exist 
        m = re.search('.*\\u2026([^\)]*)\)$', choice)
        if m: 
            choice = choice.replace(m.group(1), '') 
        if choice in canonical_choices: 
            return canonical_choices[choice] 
        return choice 
    
    df_votium_frax.bribe_choice = df_votium_frax.bribe_choice.apply(preprocess_choice)
    
    # Ensure that the same canonical choice doesn't have two different choice indices in a given round 
    numeric_cols = ['bribe_amount', 'bribe_amount_usd']
    group_cols = [c for c in df_votium_frax.columns if c not in numeric_cols and c not in ['bribe_choice_index']]
    assert all(len(gdf) == 1 for _, gdf in df_votium_frax.groupby(group_cols))
    # Ensure that all choices across all rounds were mapped to a canonical choice 
    assert not set(df_votium_frax.bribe_choice.unique()).difference(set(df_gauges.gauge_short_name.unique()))
    
    df = df_votium_frax.merge(df_gauges, how='left', left_on='bribe_choice', right_on='gauge_short_name')
    assert all(~df.gauge_short_name.isna()) 
    
    return df 

@task
def proposals_to_choices(df_proposals): 
    # Expand the proposals table so that we have one row per combination of a proposal and a choice within that proposal. 
    df_choices = df_proposals.explode('proposal_choices').reset_index().rename(columns={'proposal_choices': 'choice'})
    df_choices['choice_index'] = df_choices.groupby('proposal_id').cumcount()
    df_choices = df_choices[['choice', 'choice_index', 'proposal_round', 'proposal_title', 'proposal_id', 'proposal_id_keccak256']]
    df_choices['choice'] = df_choices.choice.str.lower()
    return df_choices 


<>:457: DeprecationWarning: invalid escape sequence '\)'
<>:457: DeprecationWarning: invalid escape sequence '\)'
/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_21416/207801480.py:457: DeprecationWarning: invalid escape sequence '\)'
  m = re.search('.*\\u2026([^\)]*)\)$', choice)
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'query_snapshot_proposals' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_21416/207801480.py:81' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'blocking_query_snapshot_votes_for_proposal' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_21416/207801480.py:145' conflicts with another task. Consider specifying a unique `name` para

In [108]:
from prefect.client import get_client

async with get_client() as client:
    # set a concurrency limit of 10 on the 'small_instance' tag
    limit_id = await client.create_concurrency_limit(tag="network_request", concurrency_limit=3)

@flow(cache_result_in_memory=False)
async def flow_votium_votes(): 
    # COINGECKO DATA 
    # ----------------------------------------------------------------
    df_prices = query_bribe_asset_prices.submit()

    # CURVE DATA 
    # ----------------------------------------------------------------
    df_gauge_info = query_gauge_info.submit()
    
    # SNAPSHOT DATA 
    # ----------------------------------------------------------------
    df_proposals = await query_snapshot_proposals()
    df_choices = proposals_to_choices(df_proposals)
    print(f"Number of votium snapshot proposals: {len(df_proposals)}")
    proposal_ids = df_proposals.proposal_id.unique().tolist()
    # Query for data for each proposal independently. The concurrency here is limited 
    # due to constraints of the snapshot api 
    results = [blocking_query_snapshot_votes_for_proposal(pid) for pid in proposal_ids]
    results = await asyncio.gather(*results)
    # Map the proposal id to the count of votes in that proposal 
    for i in range(len(results)): 
        pid = proposal_ids[i]
        # pending votes should not count yet 
        results[i] = [r for r in results[i] if r['vp_state'] == 'final']
        computed_vote_count = len(results[i])
        actual_vote_count = df_proposals.loc[df_proposals.proposal_id == pid].vote_count.values[0]
        assert actual_vote_count == computed_vote_count, "Actual and computed vote counts did not match" 
    print("Here is the count of unique votes per each convex gauge weight snapshot proposal") 
    ddf(df_proposals[['proposal_round', 'vote_count', 'proposal_id']].sort_values('proposal_round').reset_index(drop=True))

    # 
    votes = list(chain(*results)) 
    df_votes = await process_snapshot_votes(votes) # convex gauge votes 
    
    # Join votes with the choice metadata from the snapshot proposal 
    df_votes = df_votes.merge(df_choices, how='left', on=['proposal_id', 'choice_index'], validate="m:1")
    assert not df_votes.proposal_title.isna().any() # ensures each vote was matched with a proposal 
    
    # VOTIUM DATA 
    # ----------------------------------------------------------------
    df_epoches = query_votium_epoches() 
    df_epoches = validate_proposals_vs_epoches(df_proposals, df_epoches)
    df_epoches = process_df_epoches(df_epoches) # must happen after validation to remove erroneous proposal 
    epoch_ids = df_epoches.epoch_id.unique().tolist()
    df_bribes = query_votium_bribes(epoch_ids)
    df_bribes = label_bribes(df_bribes)
    
    events_claimed = query_votium_claims()
    df_claims = process_events_claimed(events_claimed) 

    # Data Joins / Processing 
    # ----------------------------------------------------------------
    # Takes each bribe in fxs or frax and joins the name of the choice voted for, voting round, and prices for reward tokens 
    df_votium_frax = join_bribes_choices_epoches_prices(df_bribes, df_choices, df_epoches, df_prices)
    # Ensures that historical choices map to choices in most recent proposal 
    df_votium_frax = standardize_choices(df_votium_frax, df_gauge_info) 
    
    return df_proposals, df_bribes, df_choices, df_votes, df_epoches, df_prices.result(), df_votium_frax, df_gauge_info.result(), df_claims


/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/flows.py:205: UserWarning: A flow named 'flow-votium-votes' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_21416/1681643241.py:7' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [109]:
df_proposals, df_bribes, df_choices, df_votes, df_epoches, df_prices, df_votium_frax, df_gauge_info, df_claims = await flow_votium_votes()

20:59:18.224 | INFO    | prefect.engine - Created flow run 'brass-poodle' for flow 'flow-votium-votes'

20:59:18.275 | INFO    | Flow run 'brass-poodle' - Created task run 'query_snapshot_proposals-2630bb84-0' for task 'query_snapshot_proposals'

20:59:18.276 | INFO    | Flow run 'brass-poodle' - Executing 'query_snapshot_proposals-2630bb84-0' immediately...

20:59:18.288 | INFO    | Flow run 'brass-poodle' - Created task run 'query_bribe_asset_prices-f0f27a15-0' for task 'query_bribe_asset_prices'

20:59:18.289 | INFO    | Flow run 'brass-poodle' - Submitted task run 'query_bribe_asset_prices-f0f27a15-0' for execution.

...

20:59:27.929 | INFO    | Flow run 'brass-poodle' - Executing 'standardize_choices-e0fce205-0' immediately...

/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_21416/207801480.py:464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_votium_frax.bribe_choice = df_votium_frax.bribe_choice.apply(preprocess_choice)


20:59:27.966 | INFO    | Task run 'standardize_choices-e0fce205-0' - Finished in state Completed()

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/utilities/asyncutils.py:233: UserWarning: `sync` called from an asynchronous context; you should `await` the async function directly instead.
  warnings.warn(
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2b52f35e0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x29dd99fc0>


20:59:28.085 | INFO    | Flow run 'brass-poodle' - Finished in state Completed()

In [80]:
df_claims.head()

,token,account,amount,token_name,token_decimals
0,0xdBdb4d16EdA451D0503b854CF79D55697F90c8DF,0xB1748C79709f4Ba2Dd82834B8c82D4a505003f27,22525613873630572544.0,ALCX,18
1,0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0,0xB1748C79709f4Ba2Dd82834B8c82D4a505003f27,5629723701838160592896.0,FXS,18
2,0xc7283b66Eb1EB5FB86327f08e1B5816b0720212B,0xB1748C79709f4Ba2Dd82834B8c82D4a505003f27,7112161722081615020032.0,TRIBE,18
4,0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0,0xB1748C79709f4Ba2Dd82834B8c82D4a505003f27,1798772505030803849216.0,FXS,18
5,0xdBdb4d16EdA451D0503b854CF79D55697F90c8DF,0xB1748C79709f4Ba2Dd82834B8c82D4a505003f27,11370655890745188352.0,ALCX,18


In [81]:
(
    alt.Chart((
        df_votes[['proposal_round', 'choice', 'choice_vp']]
        .groupby(['proposal_round', 'choice']).sum()
        .reset_index() 
    ))
    .mark_bar()
    .encode(
        x="proposal_round:O", 
        y="choice_vp:Q", 
        color="choice:N", 
        tooltip=["choice:N", "choice_vp:Q"]
    )
    .properties(title="Convex Gauge Weight Vote", width=500) 
)

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [ ]:
color = alt.Color('gauge_short_name:N', scale=alt.Scale(range=colors_28))


def chart_bribes_historical(df): 
    return (
        alt.Chart(df)
        .transform_joinaggregate(groupby=['proposal_round'], bribe_amount_total_usd="sum(bribe_amount_usd)")
        .mark_bar()
        .encode(
            x="proposal_round:O", 
            y="bribe_amount_usd:Q", 
            color=color, 
            tooltip=[
                alt.Tooltip('epoch_end_date:T'), 
                alt.Tooltip('gauge_short_name:N'), 
                alt.Tooltip('bribe_token_name:N'),
                alt.Tooltip('bribe_amount:Q', format=",d"),
                alt.Tooltip('bribe_amount_usd:Q', format="$,d"),
                alt.Tooltip('bribe_amount_total_usd:Q', format="$,d"), 
            ]
        )
    )
    
    
def chart_bribes_last_round(df): 
    return (
        alt.Chart(df)
        .mark_arc()
        .encode(
            theta="bribe_amount_usd:Q", 
            color=color, 
            tooltip=['gauge_short_name:N', alt.Tooltip('bribe_amount_usd:Q', format="$,d")]
        )
    )


data = df_votium_frax.copy()
metapool_gauge_addrs = df_pools.loc[df_pools.pool_fraxbp_metapool == True].dropna(subset='pool_gauge').pool_gauge.unique()
data['is_metapool'] = data.gauge_address.apply(lambda addr: addr in metapool_gauge_addrs)

last_round = data.proposal_round.max()
data_last = data.loc[data.proposal_round == last_round]
data_mp = data.loc[data.is_metapool == True] 
data_mp_last = data_mp.loc[data.proposal_round == last_round]

dfp = df_proposals.copy()
dfp.proposal_end = pd.to_datetime(dfp.proposal_end)

# TODO: Bribe for FraxBP in round 26 was using cvxCRV, why is this? 
(
    (chart_bribes_historical(data) | chart_bribes_last_round(data_last)) & 
    (chart_bribes_historical(data_mp) | chart_bribes_last_round(data_mp_last)).resolve_scale(theta='independent', color="shared")
)

In [ ]:
first_row(df_pool_snaps)
first_row(df_votium_frax)

In [ ]:
# data.head()

In [ ]:
# Charts here validated by going to https://curve.fi/#/ethereum/pools
# entering "fraxbp" into the search bar and comparing the tvl of all pools in the UI 
# to the tvl in the charts seen here. 
data = (
    df_mpool_snaps
    .merge(df_pools, how='left', on='pool_address')
    .merge(df_reserves, how='left', on=['pool_address', 'date'])
)
ncols = 1 
rows = []
row = []
pool_names = data.pool_name.unique().tolist()

# Amount of usd bribes per gauge and round (identified by timestamp) 
df_bribe_gauge_round = df_votium_frax.groupby(['gauge_address', 'epoch_start_date', 'epoch_end_date'])['bribe_amount_usd'].sum().reset_index()

groups = list(data.groupby("pool_name"))
def group_sort_key(g): 
    # Sort by tvl at last time point 
    gdf = g[1]
    return -1 * gdf.loc[gdf.date == gdf.date.max(), 'reserves_usd'].sum()
groups.sort(key=group_sort_key)

for pool_name, sdf in groups: 
    sdf = sdf[[
        'pool_name', 'pool_gauge', 'date', 'reserves_usd', 'snap_tvl_usd', 
        'pool_coin_name', 'snap_vol_usd', 'snap_liq_util', 'mpool_asset_eco_tvl_usd', 
    ]]
    sdf = sdf.merge(
        df_bribe_gauge_round, 
        how='left', 
        left_on=['pool_gauge', 'date'], 
        right_on=['gauge_address', 'epoch_end_date'], 
    )
    paired_asset = [e for e in sdf.pool_coin_name.unique() if e != 'crvFRAX'][0]
    
    # For each bribe on the current pool, what is the ratio of bribe / avg_tvl_next_two_weeks mea
    dfb = sdf.loc[sdf.pool_coin_name == 'crvFRAX']
    df_ranges = (
        df_bribe_gauge_round[['epoch_start_date', 'epoch_end_date']]
        .drop_duplicates()
        .sort_values('epoch_start_date').reset_index(drop=True)
    )
    df_ranges['next_epoch_tvl'] = 0
    # map each epoch start to tvl over course of next two weeks 
    epoch_start_to_epoch_tvl = {}
    epoch_end_to_next_epoch_start = {}
    for r in df_ranges.to_dict(orient="records"): 
        s = r['epoch_start_date']
        e = r['epoch_end_date'] 
        df_ranges.loc[df_ranges.epoch_start_date == s, 'next_epoch_tvl'] = (
            dfb.loc[(dfb.date >= s) & (dfb.date < e)].snap_tvl_usd.mean()
        )
    df_ranges = df_ranges.dropna()
    
    # Match each bribe with the tvl of the pool over the next epoch 
    dfb = dfb.merge(df_ranges[['epoch_start_date', 'next_epoch_tvl']], how='left', on='epoch_start_date')
    # Compute ratio of average tvl of pool over next epoch to the bribe 
    dfb['tvl_to_bribe_ratio'] = dfb.next_epoch_tvl / dfb.bribe_amount_usd
    
    # Chart showing TVL in the pool 
    tvl_base = (
        alt.Chart(sdf)
        .transform_calculate(stack_order="datum.pool_coin_name === 'crvFRAX' ? 0 : 1")
        .transform_joinaggregate(groupby=['pool_name', 'date'], tvl_total="sum(reserves_usd)")
        .encode(x="date:T", order="stack_order:O", )
    )
    chart_tvl_area = (
        tvl_base
        .mark_area()
        .encode(
            y=alt.Y("reserves_usd:Q", axis=alt.Axis(title="TVL ($)")), 
            color=alt.Color("pool_coin_name:N", scale=alt.Scale(range=colors_28)), 
            tooltip=[
                alt.Tooltip("pool_coin_name:N"), 
                alt.Tooltip("date:Q", format='$,d'), 
                alt.Tooltip("tvl_total:Q", format='$,d'), 
            ]
        )
    )
    chart_tvl_line = (
        tvl_base
        .transform_filter("datum.pool_coin_name == 'crvFRAX'")
        .mark_line()
        .encode(y="tvl_total:Q")
    )
    chart_pool_liquidity = alt.layer(chart_tvl_area, chart_tvl_line)
    # Chart showing volume in the pool 
    vol_base = (
        alt.Chart(sdf)
        .transform_filter("datum.pool_coin_name !== 'crvFRAX'")
        .encode(x="date:T", order="stack_order:O")
    )
    chart_vol_bar = (
        vol_base
        .mark_bar()
        .encode(
            y="snap_vol_usd:Q", 
            tooltip=[
                alt.Tooltip("snap_vol_usd:Q", format='$,d'), 
            ]
        )
    )
    # Chart showing liquidity utilization in the pool 
    chart_liq_util = (
        vol_base
        .mark_line()
        .encode(
            y=alt.Y("snap_liq_util:Q", scale=alt.Scale(domain=[0,1.0], clamp=True)), 
            tooltip=[
                alt.Tooltip("snap_liq_util:Q", format='$,d'), 
            ]
        )
    )
    # Chart showing ecosystem wide tvl for non crvFRAX assets 
    chart_vol_total = (
        vol_base
        .mark_bar()
        .encode(
            y="mpool_asset_eco_tvl_usd:Q", 
            tooltip=[
                alt.Tooltip("mpool_asset_eco_tvl_usd:Q", format='$,d'), 
            ]
        )
    )
    # Chart showing ecosystem wide tvl for non crvFRAX assets 
    chart_bribes = (
        vol_base
        .mark_point()
        .encode(
            y="bribe_amount_usd:Q", 
            tooltip=[
                alt.Tooltip("bribe_amount_usd:Q", format='$,d'), 
            ]
        )
    )
    chart_bribe_ratio = (
        alt.Chart(dfb.loc[~dfb.tvl_to_bribe_ratio.isna()])
        .mark_line()
        .encode(
            x="date:T",
            y="tvl_to_bribe_ratio:Q", 
            tooltip=[
                alt.Tooltip("tvl_to_bribe_ratio:Q", format='.2f'), 
            ]
        )
    )

    pool_symbol = pool_name.split(':')[-1].strip()
    w = 250
    h = 150
    
    row.append(
        alt.hconcat(
            chart_pool_liquidity.properties(title=f"TVL: {pool_symbol}", width=w, height=h), 
            chart_liq_util.properties(title=f"Liquidity Utilization (Curve): {pool_symbol}", width=w, height=h), 
            chart_vol_bar.properties(title=f"Volume (Curve): {pool_symbol}", width=w, height=h), 
            chart_vol_total.properties(title=f"Total Volume (Ecosystem): {paired_asset}", width=w, height=h), 
            chart_bribes.properties(title=f"Votium Bribes: {pool_symbol}", width=w, height=h), 
            chart_bribe_ratio.properties(title=f"Votium Bribes Ratio: {pool_symbol}", width=w, height=h), 
            bounds='flush', 
            spacing=75
        )
        .resolve_scale(x="shared")
    )
    if len(row) == ncols: 
        rows.append(row) 
        row = []
    
rows = [
    alt.hconcat(*row) 
    .resolve_scale(x="shared")
    for row in rows
]
chart = (
    alt.vconcat(*rows)
    .resolve_scale(x="shared")
)
chart

In [ ]:
# metapool_gauge_snaps = df_pool_snaps.loc[
#     (df_pool_snaps.pool_is_metapool == True) & (~df_pool_snaps.pool_gauge.isna())
# ]
# metapool_bribes = df.loc[df.gauge_address.isin(metapool_gauge_snaps.pool_gauge)]

# df_full = (
#     pd.merge(
#         metapool_gauge_snaps, 
#         metapool_bribes,
#         how='left',
#         left_on=['snapshot_timestamp', 'pool_gauge'],
#         right_on=['timestamp', 'gauge_address'], 
#     )
# )
# miss.matrix(df_full)
# ddf(df_full.head())

In [ ]:
# (
#     alt.Chart(df_full)
#     .mark_bar()
#     .encode(
#         x="snapshot_timestamp:O", 
#         y="bribe_amount_usd:Q", 
#         color="pool_symbol:N",
#     )
# ) | (
#     alt.Chart(df_full.loc[~df_full.proposal_round.isna()])
#     .mark_bar()
#     .encode(
#         x="snapshot_timestamp:O", 
#         y="snapshot_bp_lp_all_metapools_share:Q", 
#         color="pool_symbol:N",
#     )
# )

In [ ]:
# df_liquidity.dailyPoolSnapshots_timestamp = pd.to_datetime(df_liquidity.dailyPoolSnapshots_timestamp, unit='s')
# df_liquidity = df_liquidity[
#     ['dailyPoolSnapshots_timestamp', 'dailyPoolSnapshots_reservesUSD', 'pool_address', 'pools_name', 'pools_lpToken', 'pools_coins', 'pools_coinNames']
# ]
# df_liquidity.head()

In [ ]:
# # Map each bribe to liquidty data point for it's corresponding pool 
# dff_join = pd.merge_asof(
#     df.sort_values('timestamp').reset_index(drop=True),
#     df_liquidity.sort_values('dailyPoolSnapshots_timestamp').reset_index(drop=True),
#     left_by=['gauge_address'], 
#     right_by=['pool_address'], 
#     left_on=['timestamp'], 
#     right_on=['dailyPoolSnapshots_timestamp', ], 
# )[['proposal_round', 'dailyPoolSnapshots_timestamp', 'gauge_short_name', 'bribe_fxs', 'gauge_address']]
# dff_join = dff_join.loc[~dff_join.dailyPoolSnapshots_timestamp.isna()]
# print(len(dff_join))
# dff_join.head()

In [ ]:
# df_liquidity.head()

In [ ]:
# dff_join.head()

In [ ]:
# dff = (
#     # 1. Ensure that the gauge information is attached to liquidity data 
#     df_liquidity
#     .merge(
#         dff_join[['gauge_short_name', 'gauge_address']].drop_duplicates().reset_index(drop=True), 
#         how='left', 
#         left_on='pool_address', 
#         right_on='gauge_address'
#     ) 
#     .merge(
#         dff_join[['gauge_address', 'bribe_fxs', 'proposal_round', 'dailyPoolSnapshots_timestamp']], 
#         how='left', 
#         left_on=['pool_address', 'dailyPoolSnapshots_timestamp'], 
#         right_on=['gauge_address', 'dailyPoolSnapshots_timestamp'], 
#     ) 
#     .drop(columns=['gauge_address_y'])
# ) 
# dff = dff.loc[~dff.gauge_short_name.isna()]

# # dff.bribe_fxs = dff.bribe_fxs.fillna(0)
# # miss.matrix(dff) 
# print(len(df_liquidity))
# print(len(dff))

# dff.head()

In [ ]:
# base = (
#     alt.Chart(dff)
#     .transform_calculate(stack_order="datum.pools_coinNames === 'crvFRAX' ? 0 : 1")
#     .encode(
        
#         color=alt.Color("pools_coinNames:N", scale=alt.Scale(scheme="tableau20")), 
#         tooltip=alt.Tooltip("dailyPoolSnapshots_reservesUSD:Q", format='$,d'), 
#         facet=alt.Facet('pools_symbol:N', columns=3),
#         order="stack_order:O", 
#     )
#     .resolve_scale(y="independent")
# )
# liquidity = (
#     base
#     .mark_area()
#     .encode(
#         x="dailyPoolSnapshots_timestamp:T", 
#         y="dailyPoolSnapshots_reservesUSD:Q"
#     )
#     # .resolve_scale(y="independent")
# )
# bribes = (
#     base
#     .mark_bar()
#     .encode(
#         x="dailyPoolSnapshots_timestamp:T", 
#         y="bribe_fxs:Q",
#     )
#     .resolve_scale(y="independent")
# )

# # miss.matrix(dff)

# liquidity | bribes 

In [ ]:
# ncols = 3 
# rows = []
# row = []
# gnames = dff.gauge_short_name.unique().tolist()

# max_tvl = (
#     dff.groupby(['dailyPoolSnapshots_timestamp', 'gauge_short_name'])['dailyPoolSnapshots_reservesUSD'].sum().max()
# )
# max_bribe = (
#     dff['bribe_fxs'].max() 
# )

# while gnames: 
#     gname = gnames.pop()
#     # add data for this gauge to current row 
#     sdf = dff.loc[dff.gauge_short_name == gname] 
#     base = (
#         alt.Chart(sdf)
#         .transform_calculate(stack_order="datum.pools_coinNames === 'crvFRAX' ? 0 : 1")
#         .encode(
#             x="dailyPoolSnapshots_timestamp:T", 
#             color=alt.Color("pools_coinNames:N", scale=alt.Scale(scheme="tableau20")), 
#             order="stack_order:O", 
#         )
#         .properties(width=250, height=150) 
#     )
#     liquidity = base.mark_area().encode(
#         y=alt.Y("dailyPoolSnapshots_reservesUSD:Q", scale=alt.Scale(domain=[0, max_tvl * 1.05])),
#         tooltip=alt.Tooltip("dailyPoolSnapshots_reservesUSD:Q", format='$,d'), 
#     )
#     bribes = (
#         base
#         .transform_filter("datum.pools_coinNames === 'crvFRAX'")
#         .mark_bar()
#         .encode(
#             y=alt.Y("bribe_fxs:Q", scale=alt.Scale(domain=[0, max_bribe * 1.05])),
#             tooltip=alt.Tooltip("bribe_fxs:Q", format=',d'), 
#         )
#     )
#     row.append(alt.vconcat(liquidity, bribes).resolve_scale(x="shared").resolve_axis(x='shared'))
#     if len(row) == ncols: 
#         rows.append(row) 
#         row = []
    
# rows = [alt.hconcat(*row) for row in rows]
# chart = alt.vconcat(*rows)    
# chart


In [ ]:
# sdf[~sdf.bribe_fxs.isna()].head()

# miss.matrix(sdf)
# ddf(df_liquidity.loc[df_liquidity.pools_coinNames == 'ApeUSD'].head(20))